In [2]:
import pandas as pd
import numpy as np
import json
import requests
import re
import matplotlib.pyplot as plt


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import geohash2
import pyproj
import geopandas as gpd
from shapely.geometry import Point

# Immoscout mit Selenium aufrufen und Daten scrapen

In [3]:
bundesland = [
    'bayern',
    #'hessen'
]

region = [
    'wuerzburg-kreis',
    'wuerzburg'
    #'frankfurt-am-main'
]
# https://www.immobilienscout24.de/gewerbe-flaechen/de/bayern/wuerzburg-kreis/hallenproduktion-mieten/

url = 'https://www.immobilienscout24.de/gewerbe-flaechen/de/' + bundesland[0] + '/' + region[1] + '/hallenproduktion-mieten/'

# Starten Sie den Chrome-Driver mit Selenium
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Navigieren Sie zur URL und lösen Sie das Captcha manuell
driver.get(url)
input("Bitte lösen Sie das Captcha und drücken Sie Enter, um fortzufahren...")

# Extrahieren Sie den HTML-Inhalt der Seite
html_content = driver.page_source

# Schließen Sie den Chrome-Driver
driver.quit()

# Verarbeiten Sie den HTML-Inhalt mit BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')


# Relevante Daten mit BeautifulSoup filtern und in Df laden

In [4]:
script_tags = soup.find_all('script')

for script_tag in script_tags:
    script_content = script_tag.string
    if script_content and 'tilesResult' in script_content:
        tiles_result_content = script_content
        break


# Extrahieren Sie den Inhalt des tilesResult-Objekts
if tiles_result_content:
    start_index = tiles_result_content.find('{', tiles_result_content.find('tilesResult')) + 10
    end_index = tiles_result_content.find('numberOfHits') + len('numberOfHits') -18
    tiles_result_data = tiles_result_content[start_index:end_index]
    json_content = re.sub(r'"[^"]*"', lambda m: m.group(0).replace(':', ''),  tiles_result_data)
    json_content = re.sub(r'(\w+):', r'"\1":', json_content)
    df_json = pd.read_json(json_content)
    exposes_df = pd.concat([pd.json_normalize(exp) for exp in df_json['exposes']], ignore_index=True)

/var/folders/27/swfwml_s47d2057hcqghw7dh0000gn/T/ipykernel_81963/4237904289.py:17: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_json = pd.read_json(json_content)


In [5]:
exposes_df

,id,title,addressToDisplay,streetToDisplay,zipCity,postalCode,geoGrid,pictureUrls,floorSpace,area,...,realtorProfilePath,privateCustomer,numberOfRatings,ratingAverage,commercialOnTopProductType,priority,commercializationType,showcasePlacementColor,features,showLiveVideoTourTag
0,148691523,"Würzburg-Süd, ca. 3.000 m² hochwertige Hallenf...",97084 Würzburg,,97084 Würzburg,97084,u0z555035bzq,[https//pictures.immobilienscout24.de/listings...,3.000 m²,3.000 m²,...,https//www.immobilienscout24.de/anbieter/realo...,False,-1,-1.0,EXECUTIVE,False,RENT,#EE1C25,"[INDUSTRY_HALL, RAMP]",False
1,149717337,Neubauhalle vor den Toren Würzburgs - Provisi...,97080 Würzburg,,97080 Würzburg,97080,u0z56h06y9jm,[https//pictures.immobilienscout24.de/listings...,10.000 - 20.000 m²,10.000 - 20.000 m²,...,https//www.immobilienscout24.de/anbieter/bnp-p...,False,-1,-1.0,EXECUTIVE,False,RENT,#343434,"[COMMISSION_FREE, INDUSTRY_HALL]",False
2,138309049,EXKLUSIV im Alleinauftrag Logistikflächen nahe...,97076 Würzburg,,97076 Würzburg,97076,u0z56fckzm3z,[https//pictures.immobilienscout24.de/listings...,2.950 - 6.750 m²,2.950 - 6.750 m²,...,https//www.immobilienscout24.de/anbieter/colli...,False,-1,-1.0,EXECUTIVE,False,RENT,#0C9ED9,"[COMMISSION_FREE, STORAGE_HALL, RAMP]",False
3,148350409,Lager-/Logistikhalle | | 5.700 m² | | Nähe zur...,97084 Würzburg,,97084 Würzburg,97084,u0z4ft4ysetq,[https//pictures.immobilienscout24.de/listings...,650 - 5.700 m²,650 - 5.700 m²,...,https//www.immobilienscout24.de/anbieter/jones...,False,-1,-1.0,EXECUTIVE,False,RENT,#E30613,"[MINT_CONDITION, STORAGE_AREA]",False
4,142683370,Produktions-/Lagerhallen | | 2.430 m² | | teil...,97084 Würzburg,,97084 Würzburg,97084,u0z4ghgb5gd3,[https//pictures.immobilienscout24.de/listings...,1.160 - 2.430 m²,1.160 - 2.430 m²,...,NaN,False,0,0.0,EXECUTIVE,False,RENT,#343434,"[HALL, CRANE_RUNWAY]",False
5,143460873,"1.158 m² Lagerfläche | 7,80m Deckenhöhe | 01.0...",97084 Würzburg,,97084 Würzburg,97084,u0z4fyqyugv4,[https//pictures.immobilienscout24.de/listings...,1.158 - 1.270 m²,1.158 - 1.270 m²,...,https//www.immobilienscout24.de/anbieter/jones...,False,-1,-1.0,ELEVATOR,False,RENT,#E30613,[HALL],False
6,148830964,"Halle für Veranstaltungen, Büro, Sportstätte, ...","Winterhäuser Straße 87, 97084 Würzburg",Winterhäuser Straße 87,97084 Würzburg,97084,u0z54fnd1cnx,[https//pictures.immobilienscout24.de/listings...,700 m²,700 m²,...,NaN,True,0,0.0,BASIC,True,RENT,#343434,"[COMMISSION_FREE, HALL, RAMP, AUTO_LIFT]",False
7,149311497,trockene Lagerfläche,"Klingenstrasse 8a, 97084 Würzburg",Klingenstrasse 8a,97084 Würzburg,97084,u0z5477fx69t,[],15 m²,15 m²,...,NaN,True,0,0.0,BASIC,False,RENT,#343434,"[COMMISSION_FREE, STORAGE_AREA]",False
8,149059481,Laden-/Lagerfläche 1.200qm,"Hertzstr. 4, 97076 Würzburg",Hertzstr. 4,97076 Würzburg,97076,u0z56cv3mf6z,[https//pictures.immobilienscout24.de/listings...,1.200 m²,1.200 m²,...,https//www.immobilienscout24.de/anbieter/m-und...,False,45,4.6,BASIC,False,RENT,#343434,"[WELL_KEPT, STORAGE_AREA, RAMP]",True
9,147624643,Neuwertige Top Lager- und Logistikhalle,97080 Würzburg,,97080 Würzburg,97080,u0z535x7u2hq,[https//pictures.immobilienscout24.de/listings...,5.730 m²,5.730 m²,...,https//www.immobilienscout24.de/anbieter/logiv...,False,-1,-1.0,BASIC,False,RENT,#004B96,"[FIRST_TIME_USE, HALL, RAMP]",False


In [ ]:
exposes_df.to_csv('./Immobilien_Data/Wuerzburg.csv', index=False)

# Die Geodaten ins CSR3035 Format transformieren und in einen Gdf laden

In [ ]:
def geohash_to_csr3035(geohash):
    lat, lon = geohash2.decode(geohash)
    wgs84 = pyproj.Proj(init='epsg:4326')
    csr3035 = pyproj.Proj(init='epsg:3035')
    x_csr3035, y_csr3035 = pyproj.transform(wgs84, csr3035, lon, lat)
    return x_csr3035, y_csr3035

In [ ]:
exposes_df[['x_csr3035', 'y_csr3035']] = exposes_df['geoGrid'].apply(lambda x: pd.Series(geohash_to_csr3035(x)))
exposes_df['geometry'] = exposes_df.apply(lambda row: Point(row['x_csr3035'], row['y_csr3035']), axis=1)

In [ ]:
geo_bundesländer = gpd.read_file('./GeoDaten/DE_NUTS5000.gpkg')

geo_würzburg = geo_bundesländer[geo_bundesländer['NUTS_NAME'].str.contains('Würzburg')]
geo_würzburg = geo_würzburg.to_crs('3035')

In [ ]:
exposes_gdf = gpd.GeoDataFrame(exposes_df, geometry='geometry', crs='EPSG:3035')

ax = geo_würzburg.boundary.plot(color='gray', linewidth=0.5)

# Plotte gdf_loaded auf dieselbe Achse
exposes_gdf.plot(ax=ax, color='blue', markersize=5)

# Zeige den Plot an
plt.show()